In [ ]:
# HRET Empirical Validation with Monte Carlo Re-Entry Dispersion

## 1. Setup Environment

In [ ]:
!apt update && apt install -y curl build-essential pkg-config libssl-dev
!curl https://sh.rustup.rs -sSf | sh -s -- -y
import os
import sys
import shutil
import subprocess
import json
from datetime import datetime
from pathlib import Path
from dataclasses import dataclass
os.environ['PATH'] += f":{Path.home()}/.cargo/bin"
!{sys.executable} -m pip install --upgrade pip wheel maturin seaborn ipywidgets scipy plotly kaleido
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider
from scipy.integrate import solve_ivp
sns.set(style="whitegrid")
GLOBAL_SEED = 2026
np.random.seed(GLOBAL_SEED)


In [ ]:
print('Python:', sys.version)
print('maturin:', shutil.which('maturin'))

In [ ]:
## 2. Clone Repo and Build/Install Crate

In [ ]:
repo_url = 'https://github.com/infinityabundance/dsfb.git'
workspace_root = Path.cwd().resolve()
local_crate = workspace_root if (workspace_root / 'Cargo.toml').exists() and (workspace_root / 'src' / 'lib.rs').exists() else None

if local_crate is not None:
    crate_root = local_crate
else:
    repo_root = workspace_root / 'dsfb'
    crate_root = repo_root / 'crates' / 'dsfb-hret'
    if repo_root.exists():
        shutil.rmtree(repo_root)
    subprocess.run(['git', 'clone', '--depth', '1', repo_url, str(repo_root)], check=True)

env = os.environ.copy()
env['PYO3_USE_ABI3_FORWARD_COMPATIBILITY'] = '1'
subprocess.run([
    'maturin',
    'build',
    '--release',
    '--manifest-path', str(crate_root / 'Cargo.toml'),
    '--out', str(crate_root / 'dist'),
    '--quiet',
], check=True, env=env)

wheels = sorted((crate_root / 'dist').glob('dsfb_hret-*.whl'))
if not wheels:
    raise RuntimeError('No dsfb_hret wheel produced by maturin build')

subprocess.run([
    sys.executable,
    '-m',
    'pip',
    'install',
    '--force-reinstall',
    str(wheels[-1]),
], check=True)

import importlib
dsfb_hret = importlib.import_module('dsfb_hret')
print('Imported OK from', dsfb_hret.__file__)

In [ ]:
## 3. Toy Validation: Constant Velocity with Correlated Faults
# Test HRET on simple system (eq. 1: x[n+1] = x[n] + dt*v).

In [ ]:
def toy_sim(N=500, dt=0.1, v=1.0, D_k=0.1, correlated_group=0, dist_start=200, dist_mag=5.0):
    m, g = 10, 2
    group_map = [0]*5 + [1]*5
    group_map_arr = np.array(group_map)
    rho, rho_g = 0.95, [0.9, 0.9]
    beta_k, beta_g = [1.0]*m, [1.0]*g
    k_k = [[0.5]*m]  # p=1, simple gains
    observer = dsfb_hret.HretObserver(m, g, group_map, rho, rho_g, beta_k, beta_g, k_k)
    true_x = np.cumsum(np.ones(N)*dt*v)
    hat_x = np.zeros(N); hat_x[0] = true_x[0] + np.random.randn()
    errors, weights_log = [], np.zeros((N, m))
    for n in range(1, N):
        d_k = np.random.uniform(-D_k, D_k, m)
        if dist_start <= n < dist_start + 100:
            d_k[group_map_arr == correlated_group] += dist_mag if np.random.rand() > 0.5 else -dist_mag
        y_k = true_x[n] + d_k
        r_k = y_k - hat_x[n-1]
        delta_x, weights, s_k, s_g = observer.update(r_k.tolist())
        hat_x[n] = hat_x[n-1] + dt*v + delta_x[0]
        errors.append(hat_x[n] - true_x[n])
        weights_log[n] = weights
    return true_x, hat_x, np.array(errors), weights_log

true_x, hat_x, errors, weights = toy_sim()
fig, axs = plt.subplots(2, 1, figsize=(10, 8))
axs[0].plot(true_x, label='True'); axs[0].plot(hat_x, label='Estimate'); axs[0].legend()
axs[0].set_title(f'State Estimation (RMSE: {np.sqrt(np.mean(errors**2)):.3f})')
axs[1].imshow(weights.T, aspect='auto', cmap='viridis'); axs[1].set_title('Weights Evolution')
plt.show()

In [ ]:
## 4. Re-Entry Model and HRET Fusion
# 3-DoF: [x, y, vx, vy, theta]. Exponential atmosphere, drag/gravity.

In [ ]:
def reentry_dynamics(t, state, params):
    rho0, H, g0, R, m, A, Cd = params['rho0'], params['H'], params['g0'], params['R'], params['m_vehicle'], params['A'], params['Cd']
    x, y, vx, vy, theta = state
    h = y
    g = g0 * (R / (R + h))**2 if h > 0 else g0
    v = np.hypot(vx, vy)
    v_safe = max(v, 1e-9)
    rho = rho0 * np.exp(-h / H) if h > 0 else rho0
    Fd = 0.5 * rho * v**2 * Cd * A
    ax = -(Fd / m) * (vx / v_safe)
    ay = -g - (Fd / m) * (vy / v_safe)
    dtheta_dt = -(g / v_safe) * np.cos(theta) + (v / (R + h)) * np.cos(theta)
    return np.array([vx, vy, ax, ay, dtheta_dt], dtype=float)

def propagate_state(state, dt, params):
    return state + dt * reentry_dynamics(0.0, state, params)

def build_sensor_matrix(params):
    # Two channels per state component: channels 0..4 (group 0), 5..9 (group 1)
    p = params['state_dim']
    M = params['M']
    H = np.zeros((M, p), dtype=float)
    for i in range(p):
        H[i, i] = 1.0
        H[i + p, i] = 1.0
    return H

def default_params(rho=0.95):
    p = 5
    M = 10
    k_k = np.zeros((p, M), dtype=float)
    for i in range(p):
        k_k[i, i] = 0.5
        k_k[i, i + p] = 0.5

    group_map = [0] * p + [1] * p
    return {
        'rho0': 1.225,
        'H': 11000.0,
        'g0': 9.81,
        'R': 6371000.0,
        'm_vehicle': 3000.0,
        'A': 20.0,
        'Cd': 1.0,
        'state_dim': p,
        'M': M,
        'G': 2,
        'group_map': group_map,
        'group_map_arr': np.array(group_map, dtype=int),
        'rho': rho,
        'rho_g': [0.9, 0.9],
        'beta_k': [1.0] * M,
        'beta_g': [1.0, 1.0],
        'k_k': k_k.tolist(),
        'D_k': 100.0,
        'correlated_group': 1,
        'dist_mag': 500.0,
        'plasma_start': 100,
        'plasma_end': 200,
        'initial_state': np.array([0.0, 120000.0, 7800.0, -500.0, -np.pi / 90.0], dtype=float),
        'perturbations': np.array([1000.0, 1000.0, 100.0, 100.0, np.pi / 36.0], dtype=float),
    }

@dataclass
class TrialData:
    t: np.ndarray
    true_states: np.ndarray
    sensor_noise: np.ndarray
    group_disturbance: np.ndarray
    plasma_mask: np.ndarray
    initial_estimate_offset: np.ndarray

def make_trial_data(params, seed):
    rng = np.random.default_rng(seed)
    initial_state = params['initial_state'].copy()
    initial_state += rng.uniform(-params['perturbations'], params['perturbations'])

    ground_event = lambda t, y, _params: y[1]
    ground_event.terminal = True
    ground_event.direction = -1

    sol = solve_ivp(
        reentry_dynamics,
        (0.0, 600.0),
        initial_state,
        args=(params,),
        rtol=1e-6,
        max_step=1.0,
        events=ground_event,
    )

    true_states = sol.y.T
    n_steps = len(sol.t) - 1
    sensor_noise = rng.uniform(-params['D_k'], params['D_k'], size=(n_steps, params['M']))
    group_disturbance = rng.uniform(-params['dist_mag'], params['dist_mag'], size=n_steps)
    plasma_mask = np.array([
        params['plasma_start'] <= (step + 1) < params['plasma_end']
        for step in range(n_steps)
    ])
    initial_estimate_offset = rng.uniform(-100.0, 100.0, size=params['state_dim'])

    return TrialData(
        t=sol.t,
        true_states=true_states,
        sensor_noise=sensor_noise,
        group_disturbance=group_disturbance,
        plasma_mask=plasma_mask,
        initial_estimate_offset=initial_estimate_offset,
    )

class SimpleEKF:
    def __init__(self, p, process_var=5.0, meas_var=100.0):
        self.P = np.eye(p) * 100.0
        self.Q = np.eye(p) * process_var
        self.meas_var = meas_var

    def predict(self):
        self.P = self.P + self.Q

    def update(self, residual, H):
        R = np.eye(H.shape[0]) * self.meas_var
        S = H @ self.P @ H.T + R
        K = self.P @ H.T @ np.linalg.inv(S)
        delta = K @ residual
        I = np.eye(self.P.shape[0])
        self.P = (I - K @ H) @ self.P
        return delta

def make_observer(params, mode):
    if mode == 'hret':
        return dsfb_hret.HretObserver(
            params['M'], params['G'], params['group_map'],
            params['rho'], params['rho_g'], params['beta_k'], params['beta_g'], params['k_k']
        )

    if mode == 'dsfb':
        # DSFB-style baseline: singleton groups with neutral group trust layer
        return dsfb_hret.HretObserver(
            params['M'], params['M'], list(range(params['M'])),
            params['rho'], [params['rho']] * params['M'], params['beta_k'], [0.0] * params['M'], params['k_k']
        )

    raise ValueError(f'Unsupported observer mode: {mode}')

def simulate_mode(params, trial_data, mode):
    H = build_sensor_matrix(params)
    est_state = trial_data.true_states[0] + trial_data.initial_estimate_offset
    errors = []

    if mode in ('hret', 'dsfb'):
        observer = make_observer(params, mode)
    elif mode == 'ekf':
        ekf = SimpleEKF(params['state_dim'])
    else:
        raise ValueError(f'Unknown mode: {mode}')

    for step in range(1, len(trial_data.t)):
        dt = trial_data.t[step] - trial_data.t[step - 1]
        true_state = trial_data.true_states[step]
        pred_state = propagate_state(est_state, dt, params)

        sensor_noise = trial_data.sensor_noise[step - 1].copy()
        if trial_data.plasma_mask[step - 1] and params['correlated_group'] is not None:
            sensor_noise[params['group_map_arr'] == params['correlated_group']] += trial_data.group_disturbance[step - 1]

        measurement = H @ true_state + sensor_noise
        residual = measurement - H @ pred_state

        if mode in ('hret', 'dsfb'):
            delta, _, _, _ = observer.update(residual.tolist())
            correction = np.asarray(delta, dtype=float)
        else:
            ekf.predict()
            correction = ekf.update(residual, H)

        est_state = pred_state + correction
        errors.append(np.linalg.norm(est_state - true_state))

    errors = np.asarray(errors, dtype=float)
    impact = est_state[:2] / 1000.0
    rmse = float(np.sqrt(np.mean(errors**2)))
    return rmse, impact


In [ ]:
## 5. Monte Carlo Dispersion Simulation
# x360 Monte Carlo comparison: HRET vs DSFB-style vs EKF baseline.

In [ ]:
MODE_LABEL = {
    'hret': 'HRET',
    'dsfb': 'DSFB (singleton)',
    'ekf': 'EKF baseline',
}
MODE_COLOR = {
    'hret': '#1f77b4',
    'dsfb': '#ff7f0e',
    'ekf': '#2ca02c',
}

def cep50(impacts):
    center = np.mean(impacts, axis=0)
    radial = np.linalg.norm(impacts - center, axis=1)
    return float(np.median(radial))

def find_repo_root(start_path):
    start_path = Path(start_path).resolve()
    for candidate in (start_path, *start_path.parents):
        if (candidate / '.git').exists():
            return candidate
    return start_path

def make_run_output_dir(base_dir):
    base_dir = Path(base_dir)
    base_dir.mkdir(parents=True, exist_ok=True)
    stamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    run_dir = base_dir / stamp
    suffix = 1
    while run_dir.exists():
        run_dir = base_dir / f"{stamp}_{suffix:02d}"
        suffix += 1
    run_dir.mkdir(parents=True, exist_ok=False)
    return run_dir

def run_mc(n_trials=360, rho=0.95, seed=GLOBAL_SEED, modes=('hret', 'dsfb', 'ekf')):
    params = default_params(rho=rho)
    root_rng = np.random.default_rng(seed)
    trial_seeds = root_rng.integers(0, np.iinfo(np.uint32).max, size=n_trials, dtype=np.uint32)

    metrics = {
        mode: {'rmse': [], 'impact': []}
        for mode in modes
    }

    for trial_seed in trial_seeds:
        trial = make_trial_data(params, int(trial_seed))
        for mode in modes:
            rmse, impact = simulate_mode(params, trial, mode)
            metrics[mode]['rmse'].append(rmse)
            metrics[mode]['impact'].append(impact)

    for mode in modes:
        metrics[mode]['rmse'] = np.asarray(metrics[mode]['rmse'], dtype=float)
        metrics[mode]['impact'] = np.asarray(metrics[mode]['impact'], dtype=float)

    return metrics

def summarize_metrics(metrics):
    print('Mode                     Mean RMSE [m]   Std RMSE [m]   CEP50 [km]')
    print('---------------------------------------------------------------------')
    for mode, data in metrics.items():
        rmse = data['rmse']
        impacts = data['impact']
        print(f"{MODE_LABEL[mode]:<24} {np.mean(rmse):>12.2f} {np.std(rmse):>14.2f} {cep50(impacts):>11.2f}")

def export_plotly_results(metrics, output_dir):
    output_dir = Path(output_dir)
    mode_order = list(metrics.keys())

    fig_rmse = go.Figure()
    for mode in mode_order:
        fig_rmse.add_trace(go.Histogram(
            x=metrics[mode]['rmse'],
            nbinsx=60,
            name=MODE_LABEL[mode],
            opacity=0.55,
            marker_color=MODE_COLOR[mode],
        ))
    fig_rmse.update_layout(
        barmode='overlay',
        template='plotly_white',
        title='RMSE Distribution (x360 Monte Carlo)',
        xaxis_title='RMSE [m]',
        yaxis_title='Count',
    )
    fig_rmse.write_image(str(output_dir / 'rmse_distribution_plotly.png'), width=1400, height=800, scale=2)

    fig_impact = go.Figure()
    for mode in mode_order:
        impacts = metrics[mode]['impact']
        fig_impact.add_trace(go.Scatter(
            x=impacts[:, 0],
            y=impacts[:, 1],
            mode='markers',
            name=MODE_LABEL[mode],
            marker=dict(color=MODE_COLOR[mode], size=6, opacity=0.45),
        ))
    fig_impact.update_layout(
        template='plotly_white',
        title='Impact Dispersion',
        xaxis_title='Downrange [km]',
        yaxis_title='Crossrange [km]',
    )
    fig_impact.write_image(str(output_dir / 'impact_dispersion_plotly.png'), width=1400, height=800, scale=2)

    labels = [MODE_LABEL[mode] for mode in mode_order]
    mean_rmse = [float(np.mean(metrics[mode]['rmse'])) for mode in mode_order]
    std_rmse = [float(np.std(metrics[mode]['rmse'])) for mode in mode_order]
    fig_summary = go.Figure(go.Bar(
        x=labels,
        y=mean_rmse,
        marker_color=[MODE_COLOR[mode] for mode in mode_order],
        error_y=dict(type='data', array=std_rmse, visible=True),
    ))
    fig_summary.update_layout(
        template='plotly_white',
        title='Mean RMSE by Method',
        yaxis_title='Mean RMSE [m]',
    )
    fig_summary.write_image(str(output_dir / 'rmse_summary_plotly.png'), width=1200, height=700, scale=2)

    summary = {
        mode: {
            'mean_rmse_m': float(np.mean(metrics[mode]['rmse'])),
            'std_rmse_m': float(np.std(metrics[mode]['rmse'])),
            'cep50_km': cep50(metrics[mode]['impact']),
        }
        for mode in mode_order
    }
    (output_dir / 'metrics_summary.json').write_text(json.dumps(summary, indent=2))
    np.savez(
        output_dir / 'metrics_arrays.npz',
        **{f"{mode}_rmse": metrics[mode]['rmse'] for mode in mode_order},
        **{f"{mode}_impact": metrics[mode]['impact'] for mode in mode_order},
    )

    return [
        output_dir / 'rmse_distribution_plotly.png',
        output_dir / 'impact_dispersion_plotly.png',
        output_dir / 'rmse_summary_plotly.png',
    ]

repo_root = find_repo_root(Path.cwd())
output_root = repo_root / 'output-dsfb-hret'
run_output_dir = make_run_output_dir(output_root)
print('Run output directory:', run_output_dir)

metrics = run_mc(n_trials=360, rho=0.95)
summarize_metrics(metrics)
exported_pngs = export_plotly_results(metrics, run_output_dir)
print('Exported Plotly PNG files:')
for path in exported_pngs:
    print(' -', path)

fig, axs = plt.subplots(1, 2, figsize=(14, 5))
for mode, data in metrics.items():
    sns.kdeplot(data['rmse'], ax=axs[0], fill=False, label=MODE_LABEL[mode], color=MODE_COLOR[mode])
axs[0].set_title('RMSE Distribution (x360 Monte Carlo)')
axs[0].set_xlabel('RMSE [m]')
axs[0].legend()

for mode, data in metrics.items():
    impacts = data['impact']
    axs[1].scatter(impacts[:, 0], impacts[:, 1], alpha=0.35, s=18, label=MODE_LABEL[mode], color=MODE_COLOR[mode])
axs[1].set_title('Impact Dispersion')
axs[1].set_xlabel('Downrange [km]')
axs[1].set_ylabel('Crossrange [km]')
axs[1].legend()
plt.tight_layout()
plt.show()

In [ ]:
## 6. Baselines and Sensitivity
# Compare to DSFB (singleton groups) and EKF. Sweep rho.

In [ ]:
@interact(rho=FloatSlider(min=0.80, max=0.99, step=0.01, value=0.95))
def sensitivity(rho):
    # Lighter run for interactivity; compares HRET directly against DSFB-style baseline.
    local_metrics = run_mc(n_trials=120, rho=rho, seed=GLOBAL_SEED, modes=('hret', 'dsfb'))
    summarize_metrics(local_metrics)

    fig, ax = plt.subplots(1, 1, figsize=(7, 4))
    means = [np.mean(local_metrics[m]['rmse']) for m in ('hret', 'dsfb')]
    labels = [MODE_LABEL[m] for m in ('hret', 'dsfb')]
    colors = [MODE_COLOR[m] for m in ('hret', 'dsfb')]
    ax.bar(labels, means, color=colors)
    ax.set_ylabel('Mean RMSE [m]')
    ax.set_title(f'Sensitivity at rho={rho:.2f} (120 trials)')
    plt.tight_layout()
    plt.show()